#### Setup the notebook

In [10]:
#************************************************************************************************
# Imports
#************************************************************************************************
import os
from dotenv import load_dotenv
import pandas as pd
from openai import AzureOpenAI
import json
import time

# Import the assistant tools
from tools import get_list_of_dockets

#************************************************************************************************
# Load environment variables from a .env file and set local variables
#************************************************************************************************
load_dotenv()

AZURE_OPENAI_API_ENDPOINT = os.getenv("AZURE_OPENAI_API_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
RA_ASSISTANT_ID = "asst_Ga0X4abWQFGSvfDnS0TY0LcP"


#************************************************************************************************
# Create client instance of the AzureOpenAI class
#************************************************************************************************
AOAI = AzureOpenAI(
            azure_endpoint=AZURE_OPENAI_API_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION
        )

#************************************************************************************************
# Setup utility functions
#************************************************************************************************

# Pretty printing JSON
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# Pretty printing messages
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = AOAI.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)  
        time.sleep(0.5)
    return run

def submit_message(assistant_id, thread, user_message):
    AOAI.beta.threads.messages.create(thread_id=thread.id, role="user", content=user_message)
    return AOAI.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)

def get_response(thread):
    return AOAI.beta.threads.messages.list(thread_id=thread.id, order="asc")

def create_thread_and_run(user_input):
    thread = AOAI.beta.threads.create()
    run = submit_message(RA_ASSISTANT_ID, thread, user_input)
    return thread, run



In [11]:
thread, run = create_thread_and_run("What proposed regulations from 2024 have to do with ITAR.")
run = wait_on_run(run, thread)
run.status

'requires_action'

In [5]:
show_json(run)

{'id': 'run_WK0vJoiBavhsMwwo0fAqNdmX',
 'assistant_id': 'asst_XtzJoXwHNNHT9lw9kIK9W8JY',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1714692856,
 'expires_at': 1714693456,
 'failed_at': None,
 'file_ids': [],
 'instructions': "**Persona**\nYou are an expert rulemaking assistant skilled at responding to questions about government regulations and using the regulations.gov API. You write concisely and clearly, in plain language, responding directly to users.\n\n**Situation**\nUsers will ask you questions about proposed government rules. You will use your tools and knowledge to provide thorough and well-supported answers.\n\n**Tools Available**\n### get_list_of_dockets - Use this tool to get a list of dockets from regulations.gov.\n- Parameters: \n    - agencyId: Filters results for the agency acronym specified in the value. Example: 'EPA'\n    - searchTerm: Filters results on the given term.\n    - postedDate: Filters results relative to the posted date. The value must be

In [12]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: get_list_of_dockets
Function Arguments:


{'searchTerm': 'ITAR', 'postedDate': '2024-ge'}

In [9]:
function_to_call = globals()[name]
result = function_to_call(**arguments)

print("Function Output:")
print(result)

TypeError: get_list_of_dockets() missing 3 required positional arguments: 'agencyId', 'postedDate', and 'beforePostedDate'

In [7]:
responses = get_list_of_dockets(arguments["dockets"])
print("Responses:", responses)

Responses: {'CoverPage': ['DocumentType', 'AmendmentFlag', 'DocumentPeriodEndDate', 'DocumentFiscalYearFocus', 'DocumentFiscalPeriodFocus', 'TradingSymbol', 'EntityRegistrantName', 'EntityCentralIndexKey', 'CurrentFiscalYearEndDate', 'EntityCurrentReportingStatus', 'EntityInteractiveDataCurrent', 'EntityWellKnownSeasonedIssuer', 'EntityVoluntaryFilers', 'EntityFilerCategory', 'EntitySmallBusiness', 'EntityEmergingGrowthCompany', 'EntityShellCompany', 'Security12bTitle', 'SecurityExchangeName', 'EntityFileNumber', 'EntityIncorporationStateCountryCode', 'EntityTaxIdentificationNumber', 'EntityAddressAddressLine1', 'EntityAddressCityOrTown', 'EntityAddressStateOrProvince', 'EntityAddressPostalZipCode', 'CityAreaCode', 'LocalPhoneNumber', 'DocumentAnnualReport', 'DocumentTransitionReport', 'EntityCommonStockSharesOutstanding', 'DocumentFinStmtErrorCorrectionFlag', 'EntityPublicFloat', 'IcfrAuditorAttestationFlag', 'EntityListingParValuePerShare', 'AuditorName', 'AuditorFirmId', 'AuditorLoc

In [8]:
run = AOAI.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    ],
)
show_json(run)

{'id': 'run_2zF4X0NFB0ppbXeQZ9BcSid8',
 'assistant_id': 'asst_WO0kFaMNLkyxlsaWpYYHgjJ6',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1710037497,
 'expires_at': 1710038097,
 'failed_at': None,
 'file_ids': [],
 'instructions': '**Persona**\nYou are an expert Financial Analyst skilled in analyzing 10-K filings. You write concisely and clearly, in plain language, responding directly to users.\n\n**Situation**\nUsers will ask you questions about financial analysis, market trends, investment analysis, financial reporting, economic forecasts, and business strategy. You will use your analytical tools and knowledge to provide thorough and well-supported answers.\n\n**Process**\nYOU ALWAYS FOLLOW THE PROCESS OF STEPS AND SUB-STEPS OUTLINED BELOW TO RESPOND TO USERS.\n\n- 1: Ask yourself: "Is the user request within my purview as a financial analyst?" \n\n    - When not, respectfully decline to answer.\n\n    - When so, proceed to step 2.\n\n- 2: Ask yourself: "Will actual data 

In [9]:
run = wait_on_run(run, thread)
run.status

'completed'

In [11]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: get_data_from_last_10K_filing_by_section
Function Arguments:


{'ticker': 'MSFT', 'section_path': 'Role_DisclosureEARNINGSPERSHARETables'}

In [13]:
responses = get_data_from_last_10K_filing_by_section(ticker="MSFT",section_path="Role_DisclosureEARNINGSPERSHARETables")
print("Responses:", responses)

Responses: {'ScheduleOfEarningsPerShareBasicAndDilutedTableTextBlock': '<p style="text-indent:0.0%;font-size:10.0pt;margin-top:9.0pt;font-family:Times New Roman;margin-bottom:0.0pt;text-align:justify;"><span style="background-color:rgba(0,0,0,0);color:rgba(0,0,0,1);white-space:pre-wrap;font-weight:normal;font-size:10.0pt;font-family:&quot;Arial&quot;, sans-serif;min-width:fit-content;">The components of basic and diluted EPS were as follows:</span><span style="color:rgba(0,0,0,1);white-space:pre-wrap;font-weight:normal;font-size:10.0pt;font-family:&quot;Arial&quot;, sans-serif;min-width:fit-content;"> </span></p><p style="text-indent:0.0%;font-size:10.0pt;margin-top:0.0pt;font-family:Times New Roman;margin-bottom:0.0pt;text-align:left;"><span style="white-space:pre-wrap;font-size:9.0pt;font-family:&quot;Arial&quot;, sans-serif;min-width:fit-content;">\xa0</span></p>\n      <table style="margin-left:auto;border-spacing:0;table-layout:fixed;width:100.0%;border-collapse:separate;margin-ri

In [14]:
run = AOAI.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    ],
)
show_json(run)

{'id': 'run_GIurvvEIimzBcicHLAgUgoj6',
 'assistant_id': 'asst_9yJEauP0XhTfuEWFCgZNx27x',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1709999505,
 'expires_at': 1710000105,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a highly skilled Financial Analyst AI Research Assistant, designed to provide comprehensive and accurate answers to financial and business-related questions. Your core competency lies in utilizing two functions that have access 10K data from company filings. These functions are:\n\n- get_sections_of_data_available_in_last_10K_filing(ticker)\n- get_data_from_last_10K_filing(ticker, section)\n\nWhen users ask questions, assess them through the lens of financial analysis. When you need data from SEC 10K filings follow these steps. \n\n### Step 1 ###  \n- Call the get_sections_of_data_available_in_last_10K_filing(ticker) function to retrieve the sections of data available for the company in question.\n\n### Step 2 ###\n- Review the output of 

In [10]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: What XBRL data is available for accession_number: 0000950170-23-035122
assistant: The XBRL data for accession number 0000950170-23-035122 includes a wide array of financial information structured into different segments. Below are the primary categories and some examples of the type of data points available:

- **CoverPage**: This includes details such as document type, period end date, company name (EntityRegistrantName), stock ticker symbol (TradingSymbol), shares outstanding (EntityCommonStockSharesOutstanding), and auditor information (AuditorName).

- **StatementsOfIncome**: This category covers all aspects related to the income statement, like revenues, cost of goods sold (CostOfGoodsAndServicesSold), gross profit, and various expenses like research and development (ResearchAndDevelopmentExpense), leading to the net income or loss (NetIncomeLoss).

- **BalanceSheets**: This category includes financial position statements detailing assets and liabilities, such as 